In [1]:
import pandas as pd

In [2]:
data_path = './raw_datasets/ml-10m.inter'

df = pd.read_csv(data_path, sep='\t')

In [3]:
df = df.rename(columns={"user_id:token": "session_id", "item_id:token": "item_id", "rating:float": "rating", "timestamp:float": "timestamp"})

In [4]:
df.sort_values(by=['session_id','timestamp'], ignore_index=True)

,session_id,item_id,rating,timestamp
0,1,588,5.0,838983339
1,1,231,5.0,838983392
2,1,316,5.0,838983392
3,1,329,5.0,838983392
4,1,292,5.0,838983421
...,...,...,...,...
10000049,71567,1833,3.0,912649171
10000050,71567,260,5.0,912649271
10000051,71567,1721,4.0,912649271
10000052,71567,898,4.0,912649403


In [5]:
df['tmp'] = 1.0

In [6]:
df.loc[df.rating < 4.0, 'tmp'] = 0.0

In [7]:
df = df.drop(columns='rating').rename(columns={'tmp': 'rating'})

In [10]:
df[df.rating == 1]

,session_id,item_id,timestamp,rating
0,1,122,838985046,1.0
1,1,185,838983525,1.0
2,1,231,838983392,1.0
3,1,292,838983421,1.0
4,1,316,838983392,1.0
...,...,...,...,...
10000036,71567,1805,912649010,1.0
10000040,71567,1917,912580787,1.0
10000041,71567,1920,912578247,1.0
10000048,71567,2028,912580344,1.0


In [8]:
df.to_csv("./Data/ml-10m.csv", index=False)